A Distance Unit for Genes Based on Codon Usage Bias

Think about KNN, a simple way of clustering data points using a distance unit between points, the most commonly used distance metric is euclidian distance, however, difference distance metrics affect the result of KNN drastically.

Now, we have a fasta file, and we want to cluster the genes in this fasta file, based on their CUB, we know for a fact that CUB can vary between gene groups with different conditions even in the same species, can we devise a way to cluster these genes based on metrics we get from CUB statistics? (aka, come up with a way of measuring gene distance based on CUB statistics)

Let's take a shot first, first we load the fasta file.

In [1]:
def findSequenceByID(inputFile,idType="locus_tag"):
    print ("Selected id Type: %s"%(idType))
    geneDict=dict()
    from Bio import SeqIO
    records=SeqIO.parse(inputFile, "fasta")
    cnt=0
    mySum=0
    for record in records:
        mySum+=1
        header=str(record.description)
        if idType=="raw":
            geneDict[header]=str(record.seq)
        else:
            startTargetIndex=header.find(str(idType))
            if startTargetIndex<0:
#                print ("couldn't find the target idType")
                cnt+=1
                continue
            startIndex=startTargetIndex+len(idType)+1
            idName=""
            charIndex=startIndex
            while not (header[charIndex]=="]" or header[charIndex]==","):
                idName+=header[charIndex]
                charIndex+=1
            if idName not in geneDict:
                geneDict[idName]=str(record.seq)
    print ("There are %s entries NOT found out of %s"%(cnt,mySum))
    print ("%s distinct record in %s entries"%(len(geneDict),mySum))
    return geneDict

In [2]:
targetFastaFile="Fastas/c_elegan.fasta"
geneDict=findSequenceByID(targetFastaFile,idType='Gn')


Selected id Type: Gn
There are 4617 entries NOT found out of 30167
19514 distinct record in 30167 entries


Now we need to define the function that gets CUB statistics from a gene, and convert that to a distance metric. 

In [3]:


sampleGeneSeq=geneDict[list(geneDict.keys())[0]]


def MLE_from_gene(seq):
    from Code import MLE

    MLE.deltaEtaFile="Archive/Crei_Selection.csv"
    MLE.deltaMFile="Archive/Crei_Mutation.csv"
    MLE.main()
    codonList=MLE.loadSequence(sampleGeneSeq)
    MLE_PHI_List=MLE.method4(codonList)
    return MLE_PHI_List


  

Obviously, we can not directly calculate the distance between MLE values from genes (genes have difference length, thus difference size of MLE lists), one alternative is to compare the probablity distributions of these MLE value lists using hellinger distance

The Hellinger distance for two discrete distributions $p = (p_0, \dots , p_n)$ and $q = (q_0, \dots, q_n) $ is defined as follows:
$$ H(p, q) = \frac{1}{\sqrt{2}} \sqrt{\sum_i^n (\sqrt{p_i} - \sqrt{q_i})^2}$$


In [4]:
#A quick and dirty way to compute the hellindger distance between p and q(p and q are discrete probability distributions)
def hellinger_dot(p, q):
    z = np.sqrt(p) - np.sqrt(q)
    #sum of squares of a vector z is the dot product of vector z and vector z (z@z)
    return np.sqrt(z @ z / 2)

In [9]:
import numpy as np
def listToFreqProbabilityDistribtion(MLE_PHI_List,binSize=3):
    binWidth=1/float(binSize)
    bins=[0]*binSize
    binBound=[]
    for i in range(binSize+1):
        binBound.append(i*binWidth)
    binnedList=np.digitize(MLE_PHI_List,binBound,right=True)
    binnedList=sorted(binnedList)
    freqList=[0]*binSize
    for i in binnedList:
        freqList[i-1]+=1
    return freqList
        
    
#freqList=(listToFreqProbabilityDistribtion(MLE_from_gene(sampleGeneSeq))) #this is a test sample

[130, 14, 89]


In [10]:
for geneName in geneDict:
    seq=geneDict[geneName]
    

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

